# Eat Safe, Love

## Part 1: Database and Jupyter Notebook Set Up

Import the data provided in the `establishments.json` file from your Terminal. Name the database `uk_food` and the collection `establishments`.

Within this markdown cell, copy the line of text you used to import the data from your Terminal. This way, future analysts will be able to repeat your process.

e.g.: Import the dataset with `YOUR IMPORT TEXT HERE`

In [17]:
# Import dependencies
from pymongo import MongoClient
from pprint import PrettyPrinter
import pandas as pd

In [18]:
# Create an instance of MongoClient

mongo = MongoClient(port=27017)
print(mongo.list_database_names())



['admin', 'config', 'fruits_db', 'local', 'met', 'test', 'uk_food']


In [19]:
# assign the uk_food database to a variable name
db=mongo["uk_food"]

In [10]:
# review the collections in our new database

collection_names = uk_food_db.list_collection_names()
print(collection_names)

[]


In [20]:
# review the collections in our new database
db.list_collection_names()

['establishments']

In [21]:
# review a document in the establishments collection
print(document_count+str(db.establishments.count_documents({}))+horizontal_line)
pprint(db.establishments.find_one())


NameError: name 'document_count' is not defined

In [29]:
# assign the collection to a variable

establishments = db['establishments']

## Part 2: Update the Database

1. An exciting new halal restaurant just opened in Greenwich, but hasn't been rated yet. The magazine has asked you to include it in your analysis. Add the following restaurant "Penang Flavours" to the database.

In [47]:
penang_flavours = {
    "BusinessName": "Penang Flavours",
    "BusinessType": "Restaurant/Cafe/Canteen",
    "BusinessTypeID": "",
    "AddressLine1": "146A Plumstead Rd",
    "AddressLine2": "London",
    "PostCode": "SE18 7DY",
    "Phone": "",
    "LocalAuthorityCode": "511",
    "LocalAuthorityName": "Greenwich",
    "LocalAuthorityWebSite": "http://www.royalgreenwich.gov.uk",
    "LocalAuthorityEmailAddress": "health@royalgreenwich.gov.uk",
    "scores": {
        "Hygiene": "",
        "Structural": "",
        "ConfidenceInManagement": ""
    },
    "SchemeType": "FHRS",
    "geocode": {
        "longitude": "0.08384000",
        "latitude": "51.49014200"
    },
    "RightToReply": "",
    "Distance": 4623.9723280747176,
    "NewRatingPending": True
}

# insert the document into the collection
result = collection.insert_one(penang_flavours)

In [48]:
# Insert the new restaurant into the collection

establishments.insert_one(penang_flavours)

In [50]:
# Check that the new restaurant was inserted

print(list(establishments.find({"BusinessName":"Penang Flavours"})))

[{'_id': ObjectId('641caad42ca0f2905f00c5e8'), 'BusinessName': 'Penang Flavours', 'BusinessType': 'Restaurant/Cafe/Canteen', 'BusinessTypeID': '', 'AddressLine1': '146A Plumstead Rd', 'AddressLine2': 'London', 'PostCode': 'SE18 7DY', 'Phone': '', 'LocalAuthorityCode': '511', 'LocalAuthorityName': 'Greenwich', 'LocalAuthorityWebSite': 'http://www.royalgreenwich.gov.uk', 'LocalAuthorityEmailAddress': 'health@royalgreenwich.gov.uk', 'scores': {'Hygiene': '', 'Structural': '', 'ConfidenceInManagement': ''}, 'SchemeType': 'FHRS', 'geocode': {'longitude': '0.08384000', 'latitude': '51.49014200'}, 'RightToReply': '', 'Distance': 4623.972328074718, 'NewRatingPending': True}]


2. Find the BusinessTypeID for "Restaurant/Cafe/Canteen" and return only the `BusinessTypeID` and `BusinessType` fields.

In [60]:
# Find the BusinessTypeID for "Restaurant/Cafe/Canteen" and return only the BusinessTypeID and BusinessType fields

result = collection.find_one({"BusinessType": "Restaurant/Cafe/Canteen"}, {"BusinessTypeID": 1, "BusinessType": 1, "_id": 0})
print(result)

{'BusinessType': 'Restaurant/Cafe/Canteen', 'BusinessTypeID': 1}


3. Update the new restaurant with the `BusinessTypeID` you found.

In [61]:
# update the new restaurant with the correct BusinessTypeID
db.establishments.update_one({"BusinessName":"Penang Flavours"},{'$set':{'BusinessTypeID': 1}})



In [62]:
# Confirm that the new restaurant was updated

print(list(establishments.find({"BusinessName":"Penang Flavours"})))

[{'_id': ObjectId('641caad42ca0f2905f00c5e8'), 'BusinessName': 'Penang Flavours', 'BusinessType': 'Restaurant/Cafe/Canteen', 'BusinessTypeID': '', 'AddressLine1': '146A Plumstead Rd', 'AddressLine2': 'London', 'PostCode': 'SE18 7DY', 'Phone': '', 'LocalAuthorityCode': '511', 'LocalAuthorityName': 'Greenwich', 'LocalAuthorityWebSite': 'http://www.royalgreenwich.gov.uk', 'LocalAuthorityEmailAddress': 'health@royalgreenwich.gov.uk', 'scores': {'Hygiene': '', 'Structural': '', 'ConfidenceInManagement': ''}, 'SchemeType': 'FHRS', 'geocode': {'longitude': '0.08384000', 'latitude': '51.49014200'}, 'RightToReply': '', 'Distance': 4623.972328074718, 'NewRatingPending': True}]


4. The magazine is not interested in any establishments in Dover, so check how many documents contain the Dover Local Authority. Then, remove any establishments within the Dover Local Authority from the database, and check the number of documents to ensure they were deleted.

In [68]:
# Find how many documents have LocalAuthorityName as "Dover"

count = establishments.count_documents({"LocalAuthorityName": "Dover"})
print(count)


994


In [72]:
# Delete all documents where LocalAuthorityName is "Dover"

result = establishments.delete_many({"LocalAuthorityName": "Dover"})


In [73]:
# Check if any remaining documents include Dover
print(list(establishments.find({"LocalAuthorityName":"Dover"})))

[]


In [74]:
# Check that other documents remain with 'find_one'

document = collection.find_one()


5. Some of the number values are stored as strings, when they should be stored as numbers. Use `update_many` to convert `latitude` and `longitude` to decimal numbers.

In [75]:
# Change the data type from String to Decimal for longitude

pipeline = [
    {"$addFields": {"longitude_decimal": {"$toDouble": "$geocode.longitude"}}}
]

# execute the aggregation pipeline and print the results
result = list(collection.aggregate(pipeline))
print(result)


[{'_id': ObjectId('641caad42ca0f2905f00c5e8'), 'BusinessName': 'Penang Flavours', 'BusinessType': 'Restaurant/Cafe/Canteen', 'BusinessTypeID': 1, 'AddressLine1': '146A Plumstead Rd', 'AddressLine2': 'London', 'PostCode': 'SE18 7DY', 'Phone': '', 'LocalAuthorityCode': '511', 'LocalAuthorityName': 'Greenwich', 'LocalAuthorityWebSite': 'http://www.royalgreenwich.gov.uk', 'LocalAuthorityEmailAddress': 'health@royalgreenwich.gov.uk', 'scores': {'Hygiene': '', 'Structural': '', 'ConfidenceInManagement': ''}, 'SchemeType': 'FHRS', 'geocode': {'longitude': '0.08384000', 'latitude': '51.49014200'}, 'RightToReply': '', 'Distance': 4623.972328074718, 'NewRatingPending': True, 'longitude_decimal': 0.08384}]


In [76]:
# Change the data type from String to Decimal for latitude

pipeline = [
    {"$addFields": {"latitude_decimal": {"$toDouble": "$geocode.latitude"}}}
]

# execute the aggregation pipeline and print the results
result = list(collection.aggregate(pipeline))
print(result)

[{'_id': ObjectId('641caad42ca0f2905f00c5e8'), 'BusinessName': 'Penang Flavours', 'BusinessType': 'Restaurant/Cafe/Canteen', 'BusinessTypeID': 1, 'AddressLine1': '146A Plumstead Rd', 'AddressLine2': 'London', 'PostCode': 'SE18 7DY', 'Phone': '', 'LocalAuthorityCode': '511', 'LocalAuthorityName': 'Greenwich', 'LocalAuthorityWebSite': 'http://www.royalgreenwich.gov.uk', 'LocalAuthorityEmailAddress': 'health@royalgreenwich.gov.uk', 'scores': {'Hygiene': '', 'Structural': '', 'ConfidenceInManagement': ''}, 'SchemeType': 'FHRS', 'geocode': {'longitude': '0.08384000', 'latitude': '51.49014200'}, 'RightToReply': '', 'Distance': 4623.972328074718, 'NewRatingPending': True, 'latitude_decimal': 51.490142}]


In [78]:
# Check that the coordinates are now numbers

document = collection.find_one()
print(document)


{'_id': ObjectId('641caad42ca0f2905f00c5e8'), 'BusinessName': 'Penang Flavours', 'BusinessType': 'Restaurant/Cafe/Canteen', 'BusinessTypeID': 1, 'AddressLine1': '146A Plumstead Rd', 'AddressLine2': 'London', 'PostCode': 'SE18 7DY', 'Phone': '', 'LocalAuthorityCode': '511', 'LocalAuthorityName': 'Greenwich', 'LocalAuthorityWebSite': 'http://www.royalgreenwich.gov.uk', 'LocalAuthorityEmailAddress': 'health@royalgreenwich.gov.uk', 'scores': {'Hygiene': '', 'Structural': '', 'ConfidenceInManagement': ''}, 'SchemeType': 'FHRS', 'geocode': {'longitude': '0.08384000', 'latitude': '51.49014200'}, 'RightToReply': '', 'Distance': 4623.972328074718, 'NewRatingPending': True}
